In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class SVM:
    def __init__(self, X, y, kernel_type="linear", max_iter=10000, C=1.0, tolerance=0.001):
        # parameters
        self.kernels = {
            'linear': self.kernel_linear,
            'quadratic': self.kernel_quadratic
        }
        self.kernel_type = kernel_type

        self.max_passes = max_iter # max passes
        self.C = C # regularization paramtere
        self.tol = tolerance # tolerance

        # input/training-data
        self.X = X
        self.N, self.D = self.X.shape
        self.y = y


    def fit(self):
        alpha = np.zeros((self.N, ))
        b = 0
        passes = 0
        kernel = self.kernels[self.kernel_type]
        while passes < self.max_passes:
            num_changed_alphas = 0
            for i in range(self.N):
                Ei = self.get_Ek(i, self.get_w(alpha), b)
                if(((self.y[i]*Ei) < -self.tol and alpha[i] < self.C) or ((self.y[i]*Ei) > self.tol and alpha[i] > 0)):
                    j = self.get_rnd_int(self.N-1, i)  # Get random int i~=j
                    Ej = self.get_Ek(j, self.get_w(alpha), b)

                    alpha_old = np.copy(alpha) # save old alphas

                    L,H = self.get_L_H(alpha[j], alpha[i], self.y[j], self.y[i])

                    if(L == H):
                        continue
                    eta = -kernel(self.X[i], self.X[i]) -kernel(self.X[j], self.X[j]) + 2 * kernel(self.X[i], self.X[j])
                    if(eta>=0):
                        continue
                    alpha[j] = alpha[j] - self.y[j]*((Ei - Ej)/eta)
                    alpha[j] = max(alpha[j], L)
                    alpha[j] = min(alpha[j], H)
                    if(abs(alpha[j] - alpha_old[j]) < 1e-5):
                        continue

                    alpha[i] = alpha_old[i] + self.y[i]*self.y[j] * (alpha_old[j] - alpha[j])

                    b1 = b - Ei - self.y[i]*(alpha[i] - alpha_old[i])*np.dot(self.X[i], self.X[i].T) - self.y[j]*(alpha[j] - alpha_old[j])*np.dot(self.X[i], self.X[j].T)
                    b2 = b - Ej - self.y[i]*(alpha[i] - alpha_old[i])*np.dot(self.X[i], self.X[j].T) - self.y[j]*(alpha[j] - alpha_old[j])*np.dot(self.X[j], self.X[j].T)
                    if((0 < alpha[i] and alpha[i] < self.C) and (0 < alpha[j] and alpha[j] < self.C)):
                        b = (b1 + b2)/2
                    elif((0 < alpha[i] and alpha[i] < self.C)):
                        b = b1
                    elif ((0 < alpha[j] and alpha[j] < self.C)):
                        b = b2

                    num_changed_alphas+=1

            if(num_changed_alphas == 0):
                passes+=1
            else:
                passes = 0

        self.alpha = alpha
        self.w = self.get_w(alpha)
        self.b = b

        # print(passes)

        alpha_idx = np.where(alpha > 0)[0]
        support_vectors = self.X[alpha_idx, :]
        return support_vectors, passes

    def predict(self, X):
        # kernel = self.kernels[self.kernel_type]
        # eval = np.sign(np.dot(np.multiply(self.alpha, self.y), kernel(X, X)) + self.b).astype(int)

        return np.sign(np.dot(self.w, X.T) + self.b).astype(int)

    def get_w(self, alpha):
        return np.dot(np.multiply(alpha, self.y), self.X)

    def get_rnd_int(self, n, z):
        # TODO: If does not work, use external method
        arr = np.arange(n)
        np.random.shuffle(arr)
        if arr[0] == z:
            return arr[1]
        else:
            return arr[0]

    def f(self, i, w, b):
        return np.sign(np.dot(w.T, self.X[i].T) + b).astype(int)

    def get_Ek(self, i, w, b):
        # print(np.dot(w.T, self.X[i].T) + b, self.y[i], self.X[i])
        return self.f(i,  w, b) - self.y[i]

    def get_L_H(self, alpha_j, alpha_i, y_j, y_i):
        if (y_i != y_j):
            return (max(0, alpha_j - alpha_i), min(self.C, self.C - alpha_i + alpha_j))
        else:
            return (max(0, alpha_i + alpha_j - self.C), min(self.C, alpha_i + alpha_j))
#
    #  Define kernels
    def kernel_linear(self, x1, x2):
        return np.dot(x1, x2.T)
    def kernel_quadratic(self, x1, x2):
        return (np.dot(x1, x2.T) ** 2)


In [3]:
# from sklearn import datasets
# from sklearn.model_selection import train_test_split

In [4]:
# iris = datasets.load_iris()
# X = iris.data
# y = iris.target

In [5]:
# class_chosen = 1 # only this class is chosen
# y = np.asarray([-1 if y[i]!=class_chosen else 1 for i in range(y.shape[0])])

In [6]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [7]:
# C=1.0
# epsilon=0.001
# model = SVM(X_train, y_train, C=C, tolerance=epsilon)

In [8]:
# support_vectors, iterations = model.fit()
# sv_count = support_vectors.shape[0]

In [9]:
# y_hat = model.predict(X_test)

In [10]:
def calc_acc(y, y_hat):
    idx = np.where(y_hat == 1)
    TP = np.sum(y_hat[idx] == y[idx])
    idx = np.where(y_hat == -1)
    TN = np.sum(y_hat[idx] == y[idx])
    return float(TP + TN)/len(y)

In [11]:
# acc = calc_acc(y_test, y_hat)
# acc

In [12]:
# print("Support vector count: %d" % (sv_count))
# print("bias:\t\t%.3f" % (model.b))
# print("w:\t\t" + str(model.w))
# print("accuracy:\t%.3f" % (acc))
# print("Converged after %d iterations" % (iterations))

In [13]:
from skimage.feature import hog
from skimage import data, exposure
import cv2
def load_images(folder):
#     images = [np.array([])]
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.resize(img,(20,20))
        fd, hog_image = hog(img, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, multichannel=True)
        hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
        if img is not None:
#             hog_image_rescaled = hog_image_rescaled.flatten()
#             images= np.append(images,hog_image_rescaled)
            images.append(hog_image_rescaled)
            output = np.vstack(images)
#             images=np.vstack([hog_image_rescaled, hog_image_rescaled])
    return output

In [14]:
import os
imgdata = load_images('../../../data')

In [15]:
imgdata.shape

(1000, 20)

In [16]:
labels = np.array([])
for i in range(500):
    labels = np.append(labels,1)

In [17]:
for i in range(500):
    labels = np.append(labels,-1)

In [18]:
labels.shape

(1000,)

In [19]:
C=1.0
epsilon=0.001
model = SVM(imgdata, labels.astype(int), C=C, tolerance=epsilon)

In [ ]:
support_vectors, iterations = model.fit()
sv_count = support_vectors.shape[0]

In [ ]:
alien0 = cv2.imread('../../../data/alien0.jpg')
alien0 = cv2.resize(alien0,(50,50))
fd, alien0 = hog(alien0, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, multichannel=True)
alien0 = exposure.rescale_intensity(alien0, in_range=(0, 10))

In [ ]:
alien1 = cv2.imread('../../../data/alien0.jpg')
alien1 = cv2.resize(alien1,(50,50))
fd, alien1 = hog(alien1, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, multichannel=True)
alien1 = exposure.rescale_intensity(alien1, in_range=(0, 10))

In [ ]:
alien0.shape

In [ ]:
alien1.shape

In [ ]:
ar = np.vstack([alien0,alien1])
ar.shape

In [ ]:
X_train.shape

In [ ]:
arr = [0,0,0,0]
X_train = np.vstack([X_train, arr])
X_train.shape

In [ ]:
C=1.0
epsilon=0.001
model = SVM(X_train, y_train, C=C, tolerance=epsilon)

In [ ]:
X_train

In [ ]:
import sys
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
imgdata

In [ ]:
arr = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,]
arr2 = [0,1,0,0,0,0,0,0.29226093,0,0,0,0,0,0,0,0,0,0,0,0,]
ss = np.vstack([arr, arr2])
ss.shape

In [ ]:
yy = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,]
yy = [-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,]
yy = np.vstack([arr, arr2])
yy.shape

In [ ]:
C=1.0
epsilon=0.001
model = SVM(ss, yy, C=C, tolerance=epsilon)

In [ ]:
y_hat = model.predict(ss)
y_hat